In [1]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=f84efd31c9d174a01f549cecc4fb870690f04c78dddfc6c4836f7295383c3927
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
from google.colab import drive
import os
import cv2
from google.colab.patches import cv2_imshow
import datetime
import csv
import pandas as pd
import face_recognition

In [3]:
# drive.mount('/content/drive', force_remount=True)
# input_folder = '/content/drive/MyDrive/segmentation_to_process'
# output_folder = '/content/drive/MyDrive/segmentation_output'

# now = datetime.datetime.now()
# now.isoformat()
# directory = str(now)

# dest_path = os.path.join(output_folder, directory)
# archive_path = os.path.join(dest_path, "source_files")

# os.mkdir(dest_path)
# os.mkdir(archive_path)
# metadata_path = os.path.join(dest_path, 'metadata.csv')
# header = ['title', 'year', 'issue', 'page', 'filename']

# with open(metadata_path, 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)

#     # write the header
#     writer.writerow(header)
now = datetime.datetime.now()
now.isoformat()
directory = str(now)
input_folder = 'yearbook'
output_folder = 'yearbook_output'
dest_path = os.path.join(output_folder, directory)
archive_path = os.path.join(dest_path, "source_files")

In [4]:

!pip install face_recognition pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.7 MB/s eta 0:00:00


In [10]:
import os

# Set environment variables
os.environ['MONGODB_USERNAME'] = 'x'
os.environ['MONGODB_PASSWORD'] = 'x'
os.environ['MONGODB_CLUSTER'] = 'x'

In [11]:
import os
import datetime
from pymongo import MongoClient

# Initialize input and output folders
now = datetime.datetime.now()
directory = now.isoformat()
input_folder = 'yearbook'  # Update with actual path
output_folder = 'yearbook_output'  # Update with actual path

# Create directories if they do not exist
os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# MongoDB connection details
username = os.getenv('MONGODB_USERNAME')
password = os.getenv('MONGODB_PASSWORD')
cluster_address = os.getenv('MONGODB_CLUSTER')
#cluster_address = 'x'
database_name = 'peopledata'
collection_name = 'metadata'

# Encode special characters in the password if necessary
from urllib.parse import quote_plus
encoded_password = quote_plus(password)

# Construct the connection string
connection_string = f'mongodb+srv://{username}:{encoded_password}@{cluster_address}/?retryWrites=true&w=majority'

try:
    client = MongoClient(connection_string)
    # Attempt to retrieve server information to test connection
    client.server_info()
    print("Connected to MongoDB Atlas successfully.")
except Exception as e:
    print("Failed to connect to MongoDB Atlas:", e)
    exit(1)

db = client[database_name]
collection = db[collection_name]

def segment(file, input_folder, output_folder):
    filebase = os.path.splitext(file)[0]

    try:
        title, year, issue, page = filebase.split('_')
    except ValueError:
        print(f"Filebase '{filebase}' does not split into four parts as expected.")
        return

    imagepath = os.path.join(input_folder, file)
    findFaces = face_recognition.load_image_file(imagepath)
    encodings = face_recognition.face_encodings(findFaces)

    if len(encodings) > 0:
        suffix = ".jpg"
        locations = face_recognition.face_locations(findFaces)
        for (top, right, bottom, left) in locations:
            cv2.rectangle(findFaces, (left, top), (right, bottom), (0, 255, 0), 2)

        dest_filepath = os.path.join(output_folder, filebase + suffix)
        cv2.imwrite(dest_filepath, findFaces)
        encodings = [encoding.tolist() for encoding in encodings]
        data = {
            'title': title,
            'year': year,
            'issue': issue,
            'page': page,
            'filename': filebase,
            'encodings': encodings
        }
        # Add a try-except block to catch connection errors
        try:
            collection.insert_one(data)
        except Exception as e:
            print(f"Error inserting data into MongoDB: {e}")

# List all .jpg files in the input folder
file_list = [file for file in os.listdir(input_folder) if file.lower().endswith('.jpg')]
print(len(file_list), 'files found')

# Process each file
for file in file_list:
    segment(file, input_folder, output_folder)

print("Metadata saved to MongoDB.")



Connected to MongoDB Atlas successfully.
8 files found
Metadata saved to MongoDB.
